In [58]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [59]:
#read in wine csv
wine_df=pd.read_csv("../Resources/Winedata.csv")
wine_df.head()

,id,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia 2013 VulkÃ Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [60]:
# read weather csv from S3 - which is super cool, thanks Marisa
weather_df=pd.read_csv("https://grp2weather.s3.amazonaws.com/combined_weather.csv")
weather_df.head()

,Prov_Weather,Year_Weather,Precipitation,Temperature,Timeseries
0,Alsace,1901,864.18,8.41,Historical
1,Alsace,1902,737.39,8.57,Historical
2,Alsace,1903,757.38,8.90,Historical
3,Alsace,1904,704.56,9.39,Historical
4,Alsace,1905,840.87,8.72,Historical


In [61]:
#get historical data only
weather_df=weather_df[weather_df['Timeseries']=='Historical']

In [62]:
#extract year column from title field, pop title
wine_df['Year']=wine_df['title'].str.extract(r'([1-2]\d{3})')
wine_df.pop('title')
wine_df.head()

,id,country,description,designation,points,price,province,region_1,region_2,variety,winery,Year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,White Blend,Nicosia,2013
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Portuguese Red,Quinta dos Avidagos,2011
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,2013
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Riesling,St. Julian,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,2012


In [63]:
#filter wine_df for regions we want to examine
regions=['California','Washington','Bordeaux','Tuscany','Oregon','Burgundy','Cantabria','Piedmont','Veneto','New York','Alsace','Sicily & Sardinia','Champagne']
wine_df_regions=wine_df[wine_df['province'].isin(regions)]
wine_df_regions

,id,country,description,designation,points,price,province,region_1,region_2,variety,winery,Year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,White Blend,Nicosia,2013
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,2012
6,6,Italy,"Here''s a bright, informal red that opens with...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Frappato,Terre di Giurfo,2013
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Trimbach,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
129964,129965,France,"While it''s rich, this beautiful dry wine also...",Seppi Landmann VallÃ©e Noble,90,28.0,Alsace,Alsace,NaN,Pinot Gris,Domaine RieflÃ©-Landmann,2013
129966,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Pinot Noir,Citation,2004
129967,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Domaine Gresser,2013
129968,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Pinot Gris,Domaine Marcel Deiss,2012


In [64]:
# drop records with no year
wine_df_regions.dropna(subset=['Year'],inplace=True)
wine_df_regions

/Users/kyle/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,id,country,description,designation,points,price,province,region_1,region_2,variety,winery,Year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,White Blend,Nicosia,2013
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,2012
6,6,Italy,"Here''s a bright, informal red that opens with...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Frappato,Terre di Giurfo,2013
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Trimbach,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
129964,129965,France,"While it''s rich, this beautiful dry wine also...",Seppi Landmann VallÃ©e Noble,90,28.0,Alsace,Alsace,NaN,Pinot Gris,Domaine RieflÃ©-Landmann,2013
129966,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Pinot Noir,Citation,2004
129967,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Domaine Gresser,2013
129968,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Pinot Gris,Domaine Marcel Deiss,2012


In [65]:
#drop records from before 1990
wine_df_regions=wine_df_regions[wine_df_regions['Year']>'1990']
wine_df_regions

,id,country,description,designation,points,price,province,region_1,region_2,variety,winery,Year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,White Blend,Nicosia,2013
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,2012
6,6,Italy,"Here''s a bright, informal red that opens with...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Frappato,Terre di Giurfo,2013
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Trimbach,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
129964,129965,France,"While it''s rich, this beautiful dry wine also...",Seppi Landmann VallÃ©e Noble,90,28.0,Alsace,Alsace,NaN,Pinot Gris,Domaine RieflÃ©-Landmann,2013
129966,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Pinot Noir,Citation,2004
129967,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Domaine Gresser,2013
129968,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Pinot Gris,Domaine Marcel Deiss,2012


In [66]:
#replace wine region values with matching regions from weather data
#Piedmont needs to be Piemonte
wine_df_regions.loc[wine_df_regions['province']=='Piedmont','province']="Piemonte"
wine_df_regions.loc[wine_df_regions['province']=='Sicily & Sardinia','province']="Sicilia"
wine_df_regions.loc[wine_df_regions['province']=='Bordeaux','province']="Aquitaine"
wine_df_regions.loc[wine_df_regions['province']=='Northern Spain','province']="Cantabria"
wine_df_regions.loc[wine_df_regions['province']=='Champagne','province']="Champagne-Ardenne"

wine_df_regions

/Users/kyle/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,id,country,description,designation,points,price,province,region_1,region_2,variety,winery,Year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicilia,Etna,NaN,White Blend,Nicosia,2013
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,2012
6,6,Italy,"Here''s a bright, informal red that opens with...",Belsito,87,16.0,Sicilia,Vittoria,NaN,Frappato,Terre di Giurfo,2013
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Trimbach,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
129964,129965,France,"While it''s rich, this beautiful dry wine also...",Seppi Landmann VallÃ©e Noble,90,28.0,Alsace,Alsace,NaN,Pinot Gris,Domaine RieflÃ©-Landmann,2013
129966,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Pinot Noir,Citation,2004
129967,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,GewÃ¼rztraminer,Domaine Gresser,2013
129968,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Pinot Gris,Domaine Marcel Deiss,2012


In [67]:
wine_df_regions=wine_df_regions.drop(['id','description','designation','region_1','region_2','country'],axis=1)

In [68]:
wine_df_regions=wine_df_regions.dropna(subset=['price'])

In [69]:
wine_df_regions=wine_df_regions[wine_df_regions['price']<101]

In [70]:
wine_df_regions

,points,price,province,variety,winery,Year
2,87,14.0,Oregon,Pinot Gris,Rainstorm,2013
4,87,65.0,Oregon,Pinot Noir,Sweet Cheeks,2012
6,87,16.0,Sicilia,Frappato,Terre di Giurfo,2013
7,87,24.0,Alsace,GewÃ¼rztraminer,Trimbach,2012
9,87,27.0,Alsace,Pinot Gris,Jean-Baptiste Adam,2012
...,...,...,...,...,...,...
129964,90,28.0,Alsace,Pinot Gris,Domaine RieflÃ©-Landmann,2013
129966,90,75.0,Oregon,Pinot Noir,Citation,2004
129967,90,30.0,Alsace,GewÃ¼rztraminer,Domaine Gresser,2013
129968,90,32.0,Alsace,Pinot Gris,Domaine Marcel Deiss,2012


In [71]:
variety_counts=wine_df_regions.variety.value_counts()
replace_varieties=list(variety_counts[variety_counts<150].index)
replace_varieties

['G-S-M',
 'Roussanne',
 'AlbariÃ±o',
 'Vernaccia',
 'Grenache Blanc',
 'Chenin Blanc',
 'Grillo',
 'Primitivo',
 'GrÃ¼ner Veltliner',
 'SÃ©millon',
 'Nerello Mascalese',
 'Arneis',
 'FumÃ© Blanc',
 'Cortese',
 'Muscat',
 'Cabernet Sauvignon-Syrah',
 'Alsace white blend',
 'Shiraz',
 'CarmenÃ¨re',
 'Port',
 'Rosato',
 'Tannat',
 'Marsanne',
 'Semillon-Sauvignon Blanc',
 'Sauvignon Blanc-Semillon',
 'Cabernet Sauvignon-Merlot',
 'Carricante',
 'Syrah-Grenache',
 'Prosecco',
 'Muscat Canelli',
 'Sylvaner',
 'Frappato',
 'Pinot Nero',
 'Cinsault',
 'Prugnolo Gentile',
 'Carignane',
 'AligotÃ©',
 'Lemberger',
 'Pinot Meunier',
 'Cannonau',
 'Zibibbo',
 'Syrah-Cabernet Sauvignon',
 'Inzolia',
 'Insolia',
 'Claret',
 'Charbono',
 'Grenache-Syrah',
 'Catarratto',
 'Verdelho',
 'Cabernet Blend',
 'Petite Verdot',
 'Vidal Blanc',
 'Gamay Noir',
 'Merlot-Cabernet Franc',
 'Merlot-Cabernet Sauvignon',
 'Marsanne-Roussanne',
 'Touriga Nacional',
 'Orange Muscat',
 'Baco Noir',
 'Syrah-Petite Sirah

In [72]:
wine_df_regions['variety'].replace(to_replace=replace_varieties,value="NaN",inplace=True)
wine_df_regions

,points,price,province,variety,winery,Year
2,87,14.0,Oregon,Pinot Gris,Rainstorm,2013
4,87,65.0,Oregon,Pinot Noir,Sweet Cheeks,2012
6,87,16.0,Sicilia,NaN,Terre di Giurfo,2013
7,87,24.0,Alsace,GewÃ¼rztraminer,Trimbach,2012
9,87,27.0,Alsace,Pinot Gris,Jean-Baptiste Adam,2012
...,...,...,...,...,...,...
129964,90,28.0,Alsace,Pinot Gris,Domaine RieflÃ©-Landmann,2013
129966,90,75.0,Oregon,Pinot Noir,Citation,2004
129967,90,30.0,Alsace,GewÃ¼rztraminer,Domaine Gresser,2013
129968,90,32.0,Alsace,Pinot Gris,Domaine Marcel Deiss,2012


In [73]:
wine_df_regions=wine_df_regions[wine_df_regions['variety']!='NaN']
wine_df_regions

,points,price,province,variety,winery,Year
2,87,14.0,Oregon,Pinot Gris,Rainstorm,2013
4,87,65.0,Oregon,Pinot Noir,Sweet Cheeks,2012
7,87,24.0,Alsace,GewÃ¼rztraminer,Trimbach,2012
9,87,27.0,Alsace,Pinot Gris,Jean-Baptiste Adam,2012
10,87,19.0,California,Cabernet Sauvignon,Kirkland Signature,2011
...,...,...,...,...,...,...
129964,90,28.0,Alsace,Pinot Gris,Domaine RieflÃ©-Landmann,2013
129966,90,75.0,Oregon,Pinot Noir,Citation,2004
129967,90,30.0,Alsace,GewÃ¼rztraminer,Domaine Gresser,2013
129968,90,32.0,Alsace,Pinot Gris,Domaine Marcel Deiss,2012


In [74]:
winery_vcs=wine_df_regions['winery'].value_counts()
winery_vcs

Testarossa                       217
Williams Selyem                  198
Chateau Ste. Michelle            191
Columbia Crest                   146
Kendall-Jackson                  127
                                ... 
Mandolina                          1
Palmoletino                        1
ChÃ¢teau de Potiron                1
ChÃ¢teau de Terrefort Bibonne      1
Cardinal Rule                      1
Name: winery, Length: 8506, dtype: int64

In [75]:
winery_counts=wine_df_regions.winery.value_counts()
replace_wineries=list(winery_counts[winery_counts<4].index)
len(replace_wineries)

4438

In [76]:
wine_df_regions=wine_df_regions.groupby('winery').filter(lambda x: len(x)>3)

In [77]:
weather_df['Year_Weather']=weather_df['Year_Weather'].astype(str)

In [82]:
weather_df=weather_df.drop(['Timeseries'],axis=1)

In [83]:
wine_df_regions.dtypes

points        int64
price       float64
province     object
variety      object
winery       object
Year         object
dtype: object

In [84]:
weather_df.dtypes

Prov_Weather      object
Year_Weather      object
Precipitation    float64
Temperature      float64
dtype: object

In [85]:
merged_df=wine_df_regions.merge(weather_df,left_on=['province','Year'],right_on=['Prov_Weather','Year_Weather'])
merged_df

,points,price,province,variety,winery,Year,Prov_Weather,Year_Weather,Precipitation,Temperature
0,87,14.0,Oregon,Pinot Gris,Rainstorm,2013,Oregon,2013,440.10,8.68
1,87,20.0,Oregon,Pinot Noir,Acrobat,2013,Oregon,2013,440.10,8.68
2,92,48.0,Oregon,Pinot Noir,WillaKenzie Estate,2013,Oregon,2013,440.10,8.68
3,92,70.0,Oregon,Pinot Noir,Stoller,2013,Oregon,2013,440.10,8.68
4,90,85.0,Oregon,Pinot Noir,Beacon Hill,2013,Oregon,2013,440.10,8.68
...,...,...,...,...,...,...,...,...,...,...
58846,92,85.0,Burgundy,Chardonnay,Domaine Laroche,1997,Burgundy,1997,712.75,11.51
58847,86,15.0,Aquitaine,Bordeaux-style Red Blend,Baron Philippe de Rothschild,1995,Aquitaine,1995,879.16,13.66
58848,87,29.0,Oregon,Pinot Noir,Firesteed,2003,Oregon,2003,656.21,9.43
58849,92,50.0,Tuscany,White Blend,Castello di Selvole,1996,Tuscany,1996,981.05,12.51


In [9]:
#####
wine_df_regions.to_csv("../Resources/Winedata_Regions_elseRAW.csv")

In [ ]:
wine_df_regions.columns

### Target
- points

### Variables
- province - categorical 
- region_1 - categorical 
- variety - categorical 
- winery - categorical 
- Year - categorical 

### Maybe Not Variables
- description
- designation
- price

### Definitely Not Variables
- id
- country
- region_2